# Kickstart your ZenML Projects with the ZenML Hub

The ZenML Hub is the central location for users to search, discover, and share ZenML integrations, third-party plugins, and community-contributed code. It serves as a repository of pre-built pipelines, steps, materializers, stack components, and more, allowing users to quickly implement standard use cases and build better models faster.

In this example, we will use the `langchain_qa_example` plugin, available on the ZenML Hub, which demonstrates how to build a question-answering MLOps pipeline using Langchain, LlamaIndex, and OpenAI. The plugin provides a simple pipeline and steps that fetch data from various sources, create an index, and answer queries across the corpus using a GPT-3.5 (and beyond) LLM powered by OpenAI.

To get started, let's install and initialize ZenML:

In [ ]:
!pip install zenml
!zenml init

To find out about all plugins that are currently available in the ZenML Hub, you can
browse the `Plugins` section of the ZenML dashboard

![Dashboard Plugins Page](./assets/plugin_list_dashboard.png)

Alternatively, you can use the `zenml hub list` CLI command:

In [ ]:
!zenml hub list

![zenml hub list output](./assets/plugin_list_cli.png)

As we see, there are a lot of plugins available in the ZenML Hub that we can
use to kickstart our projects. In this example, we will use the 
`langchain_qa_example` plugin to build a pipeline that can answer 
questions about the ZenML documentation.

Before installing the plugin, let's briefly check out the 
[plugin source code](https://github.com/zenml-io/zenml-hub-plugins/tree/main/src/langchain_zenml_qa_example) 
to find out what it contains. Alternatively, If you prefer to inspect the code 
in your local code editor, you can also pull the source code to your local
machine using the `zenml hub clone` command:

In [ ]:
!zenml hub clone langchain_qa_example

As we see, the plugin contains several steps, a pipeline, as well as a
`build_zenml_docs_qa_pipeline` utility function:

```python
from .pipelines import qa_pipeline, build_zenml_docs_qa_pipeline
from .steps import (
    docs_loader_step,
    DocsLoaderParameters,
    index_generator_step,
    slack_loader_step,
    SlackLoaderParameters,
    web_url_loader_step,
    WebUrlLoaderParameters,
    question_answerer_step,
    QAParameters,
)
```

Let's now install this plugin and ask it questions about ZenML!

In [ ]:
!zenml hub install langchain_qa_example -y

## Ask ZenML Questions to GPT

Using the ZenML Hub, we can very easily leverage the GPT-3.5 language model for 
answering questions related to ZenML documentation. All we need to do is set
the `OPENAI_API_KEY` environment variable and call and run the
`build_zenml_docs_qa_pipeline` function from the `langchain_qa_example` plugin:

In [ ]:
import os


# TODO: You need to set your OpenAI API key below to access GPT-3.5.
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from zenml.hub.langchain_qa_example import build_zenml_docs_qa_pipeline


pipeline = build_zenml_docs_qa_pipeline(question="What is ZenML?", load_all_paths=False).run()

## Using Your Own Docs

One of the key features of the ZenML Hub is the ability to easily customize and extend pre-built pipelines and steps to suit specific use cases. In this case, we can recreate the `qa_pipeline` and pass a custom `docs_uri` to the `docs_loader_step` to specify a custom documentation URL, allowing us to ask questions about any docs page we want:

In [ ]:
from zenml.hub.langchain_qa_example import docs_loader_step, index_generator_step, question_answerer_step, qa_pipeline, DocsLoaderParameters, QAParameters

qa_pipeline(
    document_loader=docs_loader_step(DocsLoaderParameters(docs_uri="https://docs.zenml.io")),
    index_generator=index_generator_step(),
    question_answerer=question_answerer_step(
        QAParameters(question="What is ZenML?")
    ),
).run()


## Using a Different Data Source

Let us now explore how to use a different data source in the QA pipeline. The `langchain_qa_example` plugin also
contains another `web_url_loader_step` that we can use to load data from an arbitrary webpage, which we will use below.
Alternatively, we could also use any other langchain-compatible data loading step here. See, for instance, the 
[langchain_loader_steps plugin](https://github.com/zenml-io/zenml-hub-plugins/tree/main/src/langchain_loader_steps)
or the
[llama_index_loader_steps](https://github.com/zenml-io/zenml-hub-plugins/tree/main/src/langchain_loader_steps)
plugins for some inspirations.
By leveraging the power of the ZenML Hub, we can easily adapt our ML pipeline to work with a wide range of data sources, making our workflow more versatile and effective in solving real-world problems.

In [ ]:
from zenml.hub.langchain_qa_example import web_url_loader_step, WebUrlLoaderParameters

qa_pipeline(
    document_loader=web_url_loader_step(WebUrlLoaderParameters(urls=["https://zenml.io/integrations/"])),
    index_generator=index_generator_step(),
    question_answerer=question_answerer_step(
        QAParameters(question="Name five tools that ZenML integrates with.")
    ),
).run()

## Submit Your Own Plugin

The ZenML Hub is open for community contributions and anyone with a GitHub
account can submit plugins. To find out how, check out the
[ZenML Hub Plugin Template](https://github.com/zenml-io/zenml-hub-plugin-template).

If you would like to learn more about the ZenML Hub in general, check out the
[ZenML Hub Documentation](https://docs.zenml.io/user-guide/advanced-guide/environment-management/use-the-hub) or the [ZenML Hub Launch Blog Post](https://blog.zenml.io/zenml-hub-launch).